参考官方文档：[Lunar](https://6tail.cn/calendar/api.html)

In [ ]:
from datetime import datetime
from typing import Tuple, Dict
from lunar_python import Lunar

class LunarDateTimeConverter:
    """农历日期时间转换器"""
    TIANGAN = ['甲', '乙', '丙', '丁', '戊', '己', '庚', '辛', '壬', '癸']
    DIZHI = ['子', '丑', '寅', '卯', '辰', '巳', '午', '未', '申', '酉', '戌', '亥']
    HEXAGRAM_NAMES = ['乾', '兑', '离', '震', '巽', '坎', '艮', '坤']
    HEXAGRAM_ELEMENTS = {
        '乾': '天', '兑': '泽', '离': '火', '震': '雷',
        '巽': '风', '坎': '水', '艮': '山', '坤': '地'
    }
    GUA_64 = {
    # 乾宫八卦 
    # 本宫: 乾为天, 一世: 天风姤, 二世: 天山遁, 三世: 天地否, 四世: 风地观, 五世: 山地剥, 游魂: 火地晋, 归魂: 火天大有
    '乾乾': '乾为天（六冲）',      # 乾宫本宫 (上乾下乾)
    '乾巽': '天风姤',              # 乾宫一世 (上乾下巽)
    '乾艮': '天山遁',              # 乾宫二世 (上乾下艮)
    '乾坤': '天地否（六合）',      # 乾宫三世 (上乾下坤)
    '巽坤': '风地观',              # 乾宫四世 (上巽下坤)
    '艮坤': '山地剥',              # 乾宫五世 (上艮下坤)
    '离坤': '火地晋',              # 乾宫游魂 (上离下坤)
    '离乾': '火天大有',            # 乾宫归魂 (上离下乾)
    
    # 坎宫八卦
    # 本宫: 坎为水, 一世: 水泽节, 二世: 水雷屯, 三世: 水火既济, 四世: 泽火革, 五世: 雷火丰, 游魂: 地火明夷, 归魂: 地水师
    '坎坎': '坎为水（六冲）',      # 坎宫本宫 (上坎下坎)
    '坎兑': '水泽节（六合）',      # 坎宫一世 (上坎下兑)
    '坎震': '水雷屯',              # 坎宫二世 (上坎下震)
    '坎离': '水火既济',            # 坎宫三世 (上坎下离)
    '兑离': '泽火革',              # 坎宫四世 (上兑下离)
    '震离': '雷火丰',              # 坎宫五世 (上震下离)
    '坤离': '地火明夷',            # 坎宫游魂 (上坤下离)
    '坤坎': '地水师',              # 坎宫归魂 (上坤下坎)
    
    # 艮宫八卦
    # 本宫: 艮为山, 一世: 山火贲, 二世: 山天大畜, 三世: 山泽损, 四世: 火泽睽, 五世: 天泽履, 游魂: 风泽中孚, 归魂: 风山渐
    '艮艮': '艮为山（六冲）',        # 艮宫本宫 (上艮下艮)
    '艮离': '山火贲（六合）',        # 艮宫一世 (上艮下离)
    '艮乾': '山天大畜',            # 艮宫二世 (上艮下乾)
    '艮兑': '山泽损',              # 艮宫三世 (上艮下兑)
    '离兑': '火泽睽',              # 艮宫四世 (上离下兑)
    '乾兑': '天泽履',              # 艮宫五世 (上乾下兑)
    '巽兑': '风泽中孚（六合）',    # 艮宫游魂 (上巽下兑)
    '巽艮': '风山渐',              # 艮宫归魂 (上巽下艮)
    
    # 震宫八卦
    # 本宫: 震为雷, 一世: 雷地豫, 二世: 雷水解, 三世: 雷风恒, 四世: 地风升, 五世: 水风井, 游魂: 泽风大过, 归魂: 泽雷随
    '震震': '震为雷（六冲）',        # 震宫本宫 (上震下震)
    '震坤': '雷地豫（六合）',        # 震宫一世 (上震下坤)
    '震坎': '雷水解',              # 震宫二世 (上震下坎)
    '震巽': '雷风恒',              # 震宫三世 (上震下巽)
    '坤巽': '地风升',              # 震宫四世 (上坤下巽)
    '坎巽': '水风井',              # 震宫五世 (上坎下巽)
    '兑巽': '泽风大过',            # 震宫游魂 (上兑下巽)
    '兑震': '泽雷随',              # 震宫归魂 (上兑下震)
    
    # 巽宫八卦
    # 本宫: 巽为风, 一世: 风天小畜, 二世: 风火家人, 三世: 风雷益, 四世: 天雷无妄, 五世: 火雷噬嗑, 游魂: 山雷颐, 归魂: 山风蛊
    '巽巽': '巽为风（六冲）',        # 巽宫本宫 (上巽下巽)
    '巽乾': '风天小畜',            # 巽宫一世 (上巽下乾)
    '巽离': '风火家人',            # 巽宫二世 (上巽下离)
    '巽震': '风雷益',              # 巽宫三世 (上巽下震)
    '乾震': '天雷无妄（六冲）',    # 巽宫四世 (上乾下震)
    '离震': '火雷噬嗑',            # 巽宫五世 (上离下震)
    '艮震': '山雷颐',              # 巽宫游魂 (上艮下震)
    '艮巽': '山风蛊',              # 巽宫归魂 (上艮下巽)
    
    # 离宫八卦
    # 本宫: 离为火, 一世: 火山旅, 二世: 火风鼎, 三世: 火水未济, 四世: 山水蒙, 五世: 风水涣, 游魂: 天水讼, 归魂: 天火同人
    '离离': '离为火（六冲）',        # 离宫本宫 (上离下离)
    '离艮': '火山旅（六合）',        # 离宫一世 (上离下艮)
    '离巽': '火风鼎',              # 离宫二世 (上离下巽)
    '离坎': '火水未济',            # 离宫三世 (上离下坎)
    '艮坎': '山水蒙',              # 离宫四世 (上艮下坎)
    '巽坎': '风水涣',              # 离宫五世 (上巽下坎)
    '乾坎': '天水讼',              # 离宫游魂 (上乾下坎)
    '乾离': '天火同人',            # 离宫归魂 (上乾下离)
    
    # 坤宫八卦
    # 本宫: 坤为地, 一世: 地雷复, 二世: 地泽临, 三世: 地天泰, 四世: 雷天大壮, 五世: 泽天夬, 游魂: 水天需, 归魂: 水地比
    '坤坤': '坤为地（六冲）',        # 坤宫本宫 (上坤下坤)
    '坤震': '地雷复（六合）',        # 坤宫一世 (上坤下震)
    '坤兑': '地泽临',              # 坤宫二世 (上坤下兑)
    '坤乾': '地天泰（六合）',        # 坤宫三世 (上坤下乾)
    '震乾': '雷天大壮（六冲）',    # 坤宫四世 (上震下乾)
    '兑乾': '泽天夬',              # 坤宫五世 (上兑下乾)
    '坎乾': '水天需',              # 坤宫游魂 (上坎下乾)
    '坎坤': '水地比',              # 坤宫归魂 (上坎下坤)
    
    # 兑宫八卦
    # 本宫: 兑为泽, 一世: 泽水困, 二世: 泽地萃, 三世: 泽山咸, 四世: 水山蹇, 五世: 地山谦, 游魂: 雷山小过, 归魂: 雷泽归妹
    '兑兑': '兑为泽（六冲）',        # 兑宫本宫 (上兑下兑)
    '兑坎': '泽水困（六合）',        # 兑宫一世 (上兑下坎)
    '兑坤': '泽地萃',              # 兑宫二世 (上兑下坤)
    '兑艮': '泽山咸',              # 兑宫三世 (上兑下艮)
    '坎艮': '水山蹇',              # 兑宫四世 (上坎下艮)
    '坤艮': '地山谦',              # 兑宫五世 (上坤下艮)
    '震艮': '雷山小过',            # 兑宫游魂 (上震下艮)
    '震兑': '雷泽归妹'             # 兑宫归魂 (上震下兑)
    }

    def __init__(self):
        self.lunar = Lunar.fromDate(datetime.now())
        
    def get_basic_info(self) -> Dict:
        """获取基础农历信息"""
        return {
            'year': self.lunar.getYear(),
            'month': self.lunar.getMonth(),
            'day': self.lunar.getDay(),
            'hour': self.lunar.getHour(),
            'minute': self.lunar.getMinute()
        }
    
    def get_ganzhi_info(self) -> Dict:
        """获取干支信息"""
        return {
            'year_gz': self.lunar.getYearInGanZhiExact(),
            'month_gz': self.lunar.getMonthInGanZhiExact(),
            'day_gz': self.lunar.getDayInGanZhi(),
            'hour_gz': self.lunar.getTimeInGanZhi()
        }

class HexagramCalculator:
    """卦象计算器"""
    @staticmethod
    def get_tiangan_number(gan: str) -> int:
        """获取天干序号"""
        return LunarDateTimeConverter.TIANGAN.index(gan) + 1
    
    @staticmethod
    def get_dizhi_number(zhi: str, is_month: bool = False) -> int:
        """获取地支序号"""
        base = LunarDateTimeConverter.DIZHI.index(zhi) + 1
        return (base + 10) % 12 + 1 if is_month else base
    
    @staticmethod
    def calculate_hexagram_indices(parameters: Dict) -> Tuple[int, int, int]:
        """计算卦象索引"""
        sum_base = parameters['dizhi_year'] + parameters['month'] + parameters['day']
        upper = sum_base % 8 or 8
        lower = (sum_base + parameters['dizhi_hour']) % 8 or 8
        moving_line = (sum_base + parameters['dizhi_hour']) % 6 or 6
        return upper, lower, moving_line
    
    @staticmethod
    def generate_binary_representation(number: int, length: int = 3) -> str:
        """生成二进制表示"""
        return bin(number)[2:].zfill(length)[::-1]
    
    @classmethod
    def calculate_changes(cls, upper: int, lower: int, moving_line: int) -> Tuple[str, str]:
        """计算本卦变卦"""
        ben_gua = cls.generate_binary_representation(upper-1) + cls.generate_binary_representation(lower-1)
        bian_list = list(ben_gua)
        bian_list[-moving_line] = str(1 - int(bian_list[-moving_line]))
        return ben_gua, "".join(bian_list)

class ResultFormatter:
    """结果格式化输出"""
    @staticmethod
    def format_output(basic_info: Dict, ganzhi_info: Dict, hexagram_data: Dict) -> str:
        """格式化输出结果"""
        output = []
        output.append("基础农历信息:")
        for k, v in basic_info.items():
            output.append(f"{k:8}: {v}")
        
        output.append("\n干支信息:")
        for k, v in ganzhi_info.items():
            output.append(f"{k:8}: {v}")
        
        output.append("\n卦象信息:")
        hex_items = [
            ('上卦', 'upper_hexagram'), ('下卦', 'lower_hexagram'),
            ('动爻', 'moving_line'), ('本卦', 'ben_gua'), ('变卦', 'bian_gua')
        ]
        for label, key in hex_items:
            output.append(f"{label:8}: {hexagram_data[key]}")
        
        return "\n".join(output)

def main():
    # 初始化核心组件
    converter = LunarDateTimeConverter()
    calculator = HexagramCalculator()
    
    # 获取基础数据
    basic_info = converter.get_basic_info()
    ganzhi_info = converter.get_ganzhi_info()
    
    # 准备计算参数
    calculation_params = {
        'dizhi_year': HexagramCalculator.get_dizhi_number(ganzhi_info['year_gz'][1]),
        'month': basic_info['month'],
        'day': basic_info['day'],
        'dizhi_hour': HexagramCalculator.get_dizhi_number(ganzhi_info['hour_gz'][1])
    }
    
    # 计算卦象信息
    upper_idx, lower_idx, moving_line = calculator.calculate_hexagram_indices(calculation_params)
    
    # 生成卦象数据
    ben_gua, bian_gua = calculator.calculate_changes(upper_idx, lower_idx, moving_line)
    hexagram_data = {
        'upper_hexagram': f"{converter.HEXAGRAM_NAMES[upper_idx-1]} {calculator.generate_binary_representation(upper_idx-1)}",
        'lower_hexagram': f"{converter.HEXAGRAM_NAMES[lower_idx-1]} {calculator.generate_binary_representation(lower_idx-1)}",
        'moving_line': moving_line,
        'ben_gua': ben_gua,
        'bian_gua': bian_gua
    }
    
    # 格式化输出结果
    print(ResultFormatter.format_output(basic_info, ganzhi_info, hexagram_data))

if __name__ == "__main__":
    main()

基础农历信息:
year    : 2025
month   : 4
day     : 10
hour    : 10
minute  : 58

干支信息:
year_gz : 乙巳
month_gz: 辛巳
day_gz  : 丙子
hour_gz : 癸巳

卦象信息:
上卦      : 震 110
下卦      : 兑 100
动爻      : 2
本卦      : 110100
变卦      : 110110


In [ ]:
from wcwidth import wcswidth  # 需要安装wcwidth库：pip install wcwidth

def format_gua(ben_gua, bian_gua, ben_yao_info, bian_yao_info):
    yang = '■■■■■'  # 阳爻
    yin = '■■ ■■'    # 阴爻
    
    # 生成行内容
    def build_lines(gua, yao_info, is_ben=True):
        lines = []
        for idx in reversed(range(6)):  # 从初爻到上爻转为输出时的上爻到初爻
            yao = gua['yao'][idx]
            info = yao_info[idx]
            
            # 构建本卦行
            if is_ben:
                symbols = f"{info['liu_shen']} {yang if yao == '1' else yin}"
                name = info['name']
                line = f"{symbols} {name}"
                if info.get('dong', False):
                    dong_symbol = '○→' if yao == '1' else 'x→'
                    line += f" {dong_symbol}"
            # 构建变卦行
            else:
                line = f"{yang if yao == '1' else yin} {info['name']}"
            lines.append(line)
        return lines

    # 构建内容
    ben_lines = build_lines(ben_gua, ben_yao_info, is_ben=True)
    bian_lines = build_lines(bian_gua, bian_yao_info, is_ben=False)

    # 计算显示宽度（考虑全角字符）
    def get_max_width(lines):
        return max(wcswidth(line) for line in lines) + 2

    ben_width = get_max_width(ben_lines)
    bian_width = get_max_width(bian_lines)

    # 生成对齐的标题
    ben_title = f"{ben_gua['gong']}：{ben_gua['name']}"
    bian_title = f"{bian_gua['gong']}：{bian_gua['name']}"
    print(f"{ben_title.ljust(ben_width)}{bian_title}")

    # 对齐列头
    print(f"{'六神 【本 卦】'.ljust(ben_width)}【变 卦】")

    # 对齐每行内容
    for bline, b_line in zip(ben_lines, bian_lines):
        # 计算需要填充的空格数
        fill_space = ' ' * (ben_width - wcswidth(bline))
        print(f"{bline}{fill_space}{b_line}")

# 使用示例数据测试
ben_gua = {
    'gong': '乾宫',
    'name': '乾为天（六冲）',
    'yao': '111111'
}

bian_gua = {
    'gong': '巽宫',
    'name': '风天小畜',
    'yao': '110111'
}

ben_yao_info = [
    {'liu_shen':'朱雀', 'name':'父母戌土 世', 'dong':False},
    {'liu_shen':'青龙', 'name':'兄弟申金', 'dong':False},
    {'liu_shen':'玄武', 'name':'官鬼午火', 'dong':True},
    {'liu_shen':'白虎', 'name':'父母辰土 应', 'dong':False},
    {'liu_shen':'腾蛇', 'name':'妻财寅木', 'dong':False},
    {'liu_shen':'勾陈', 'name':'子孙子水', 'dong':False}
]

bian_yao_info = [
    {'name':'子孙子水 世'},
    {'name':'妻财寅木'},
    {'name':'父母辰土'},
    {'name':'父母未土 应'},
    {'name':'官鬼巳火'},
    {'name':'妻财卯木'}
]

乾宫：乾为天（六冲）              巽宫：风天小畜
六神 【本 卦】                【变 卦】
勾陈 ■■■■■ 子孙子水     ■■■■■ 妻财卯木
腾蛇 ■■■■■ 妻财寅木     ■■■■■ 官鬼巳火
白虎 ■■■■■ 父母辰土 应  ■■■■■ 父母未土 应
玄武 ■■■■■ 官鬼午火 ○→  ■■ ■■ 父母辰土
青龙 ■■■■■ 兄弟申金     ■■■■■ 妻财寅木
朱雀 ■■■■■ 父母戌土 世  ■■■■■ 子孙子水 世


In [ ]:
{
  "output": [
    "{\"评分指标\": \"1 项目服务需求响应情况\", \"指标要求\": \"根据投标人对‘服务内容’等章节内容的响应情况进行评审： （1）完全满足并优于采购需求，得10分； （2）完全满足采购需求，得6分； （3）不能完全满足采购需求，得3分； （4）未提供响应表或方案的不得分。 注：须提供项目服务需求响应方案及《技术和服务要求响应表》，响应表中未注明偏离情况视为未响应。 根据投标人对以下项目服务需求的响应情况进行评审：1.针对15个三级和二级等保备案名称的系统进行网络安全等级保护测评服务，包括但不限于差距评估、整改建议、再次测评及出具报告；2.服务团队不少于5人，其中需具备信息安全等级测评师证书（高级）或网络安全等级测评师证书，并提供社保证明或承诺；3.使用符合要求的测评工具，确保无知识产权纠纷；4.交付包括《等级保护测评实施方案》、《系统问题清单及整改建议》、《网络安全等级保护测评报告》等文档；5.服务质量保证措施、实施进度计划（含各系统单独测试计划）及风险控制措施；6.服务响应时间要求4小时内，紧急情况30分钟内；7.保密协议及保密措施。投标人需在响应文件中详细说明如何满足上述要求，并提供相应证明材料或承诺函。\"}",
    "{\"评分指标\": \"2 等级保护测评方案\", \"指标要求\": \"（1）方案完全满足并优于采购需求，得10分； （2）方案完全满足采购需求，得6分； （3）方案不能完全满足采购需求，得3分； （4）未提供方案的不得分。根据项目需求，等级保护测评方案需严格按照GB/T28448-2019《信息安全技术网络安全等级保护测评要求》开展，涵盖以下内容：1）测评对象包括多个三级和二级信息系统，需明确各系统的测评要求；2）服务内容需包括差距评估、整改建议、再次测评及提交符合公安部要求的测评报告；3）技术要求覆盖安全物理环境、通信网络、区域边界、计算环境及管理中心等层面；4）服务人员需具备相应资格证书且团队不少于5人；5）测评工具需自行提供并确保无知识产权纠纷；6）交付成果需包含测评实施方案、问题清单、整改建议及最终测评报告；7）服务质量保证措施、详细进度计划及风险管控方案；8）响应时间要求4小时内，紧急情况30分钟内；9）严格保密协议。\"}",
    "",
    "{\"评分指标\": \"4 测评工具能力\", \"指标要求\": \"根据投标人提供的测评工具进行评审： （1）信息安全等级保护测评软件系统。 （2）web应用源代码扫描软件系统。 （3）漏洞信息检测系统。 （4）移动应用安全漏洞及合规检测系统或APP检测系统。 （5）数据库安全检查工具。 投标人每具备一个工具得3分，本小项最高得15分. 注：如为自研的，须提供国家版权局出具的有效软件著作权登记证书；购买的，须提供购买发票；如为租赁，需提供有效期内的租赁证明或软件授权书。以上均不提供或均不满足要求的不得分。\"}"
  ]
}

In [14]:
# --- 数据定义 ---
TIANGAN = ['甲', '乙', '丙', '丁', '戊', '己', '庚', '辛', '壬', '癸']
DIZHI = ['子', '丑', '寅', '卯', '辰', '巳', '午', '未', '申', '酉', '戌', '亥']
ELEMENTS = ['木', '火', '土', '金', '水']

TIANGAN_INFO = {
    '甲': {'yin_yang': '阳', 'element': '木'}, '乙': {'yin_yang': '阴', 'element': '木'},
    '丙': {'yin_yang': '阳', 'element': '火'}, '丁': {'yin_yang': '阴', 'element': '火'},
    '戊': {'yin_yang': '阳', 'element': '土'}, '己': {'yin_yang': '阴', 'element': '土'},
    '庚': {'yin_yang': '阳', 'element': '金'}, '辛': {'yin_yang': '阴', 'element': '金'},
    '壬': {'yin_yang': '阳', 'element': '水'}, '癸': {'yin_yang': '阴', 'element': '水'},
}
DIZHI_INFO = {
    '子': {'yin_yang': '阳', 'element': '水'}, '丑': {'yin_yang': '阴', 'element': '土'},
    '寅': {'yin_yang': '阳', 'element': '木'}, '卯': {'yin_yang': '阴', 'element': '木'},
    '辰': {'yin_yang': '阳', 'element': '土'}, '巳': {'yin_yang': '阴', 'element': '火'},
    '午': {'yin_yang': '阳', 'element': '火'}, '未': {'yin_yang': '阴', 'element': '土'},
    '申': {'yin_yang': '阳', 'element': '金'}, '酉': {'yin_yang': '阴', 'element': '金'},
    '戌': {'yin_yang': '阳', 'element': '土'}, '亥': {'yin_yang': '阴', 'element': '水'},
}

# 八卦定义 (爻由下至上, 1为阳, 0为阴)
TRIGRAMS = {
    '乾': {'lines': (1, 1, 1), 'element': '金', 'char': '☰'},
    '兑': {'lines': (0, 1, 1), 'element': '金', 'char': '☱'},
    '离': {'lines': (1, 0, 1), 'element': '火', 'char': '☲'},
    '震': {'lines': (0, 0, 1), 'element': '木', 'char': '☳'},
    '巽': {'lines': (1, 1, 0), 'element': '木', 'char': '☴'},
    '坎': {'lines': (0, 1, 0), 'element': '水', 'char': '☵'},
    '艮': {'lines': (1, 0, 0), 'element': '土', 'char': '☶'},
    '坤': {'lines': (0, 0, 0), 'element': '土', 'char': '☷'},
}

# 六十四卦卦名 (部分示例，实际应完整)
# 键为6爻二进制字符串 (初爻在最右)
HEXAGRAM_NAMES = {
    "111111": "乾为天", "000000": "坤为地", "100000": "地雷复", "011111": "天风姤",
    #... 更多卦名
}

# 八宫定义 (卦的二进制表示：上卦三位 + 下卦三位，例如 乾为天 111111)
# '世序' (generation_type) 用于确定世爻
BA_GONG_PALACES = {
    '乾': {'head_trigram': '乾', 'element': '金', 'hexagrams': {
        "111111": {"name": "乾为天", "generation_type": "八纯"}, # 乾上乾下
        "011111": {"name": "天风姤", "generation_type": "一世"}, # 乾上巽下 (巽初爻0)
        "101111": {"name": "天山遁", "generation_type": "二世"}, # 乾上艮下 (艮二爻0)
        "000111": {"name": "天地否", "generation_type": "三世"}, # 乾上坤下 (坤三爻0)
        "000110": {"name": "风地观", "generation_type": "四世"}, # 巽上坤下
        "000100": {"name": "山地剥", "generation_type": "五世"}, # 艮上坤下
        "000101": {"name": "火地晋", "generation_type": "游魂"}, # 离上坤下
        "111101": {"name": "火天大有", "generation_type": "归魂"}  # 离上乾下
    }},
    '坤': {'head_trigram': '坤', 'element': '土', 'hexagrams': {
        "000000": {"name": "坤为地", "generation_type": "八纯"},
        "100000": {"name": "地雷复", "generation_type": "一世"},
        "010000": {"name": "地泽临", "generation_type": "二世"},
        "111000": {"name": "地天泰", "generation_type": "三世"},
        "111001": {"name": "雷天大壮", "generation_type": "四世"},
        "111011": {"name": "泽天夬", "generation_type": "五世"},
        "111010": {"name": "水天需", "generation_type": "游魂"},
        "000010": {"name": "水地比", "generation_type": "归魂"}
    }},
    #... 其他六宫 (震巽坎离艮兑) 定义类似
}
# 简化的八宫，实际应用需要补全所有64卦的归属
# 震宫 (木): 震、豫、解、恒、升、井、大过、随
# 巽宫 (木): 巽、小畜、家人、益、无妄、噬嗑、颐、蛊
# 坎宫 (水): 坎、节、屯、既济、革、丰、明夷、师
# 离宫 (火): 离、旅、鼎、未济、蒙、涣、讼、同人
# 艮宫 (土): 艮、贲、大畜、损、睽、履、中孚、渐
# 兑宫 (金): 兑、困、萃、咸、蹇、谦、小过、归妹

# 纳甲规则 (参照表3)
# (天干, 地支) 顺序：初爻到上爻
NAJIA_RULES = {
    '乾': [('甲', '子'), ('甲', '寅'), ('甲', '辰'), ('壬', '午'), ('壬', '申'), ('壬', '戌')],
    '坤': [('乙', '未'), ('乙', '巳'), ('乙', '卯'), ('癸', '丑'), ('癸', '亥'), ('癸', '酉')],
    '震': [('庚', '子'), ('庚', '寅'), ('庚', '辰'), ('庚', '午'), ('庚', '申'), ('庚', '戌')],
    '巽': [('辛', '丑'), ('辛', '亥'), ('辛', '酉'), ('辛', '未'), ('辛', '巳'), ('辛', '卯')],
    '坎': [('戊', '寅'), ('戊', '辰'), ('戊', '午'), ('戊', '申'), ('戊', '戌'), ('戊', '子')],
    '离': [('己', '卯'), ('己', '丑'), ('己', '亥'), ('己', '酉'), ('己', '未'), ('己', '巳')],
    '艮': [('丙', '辰'), ('丙', '午'), ('丙', '申'), ('丙', '戌'), ('丙', '子'), ('丙', '寅')],
    '兑': [('丁', '巳'), ('丁', '卯'), ('丁', '丑'), ('丁', '亥'), ('丁', '酉'), ('丁', '未')],
}

# 世应规则 (参照表4)
# 爻位从1 (初爻) 到 6 (上爻)
SHI_YING_RULES = {
    "八纯": {"shi": 6, "ying": 3},
    "一世": {"shi": 1, "ying": 4},
    "二世": {"shi": 2, "ying": 5},
    "三世": {"shi": 3, "ying": 6},
    "四世": {"shi": 4, "ying": 1},
    "五世": {"shi": 5, "ying": 2},
    "游魂": {"shi": 4, "ying": 1}, # 京房八宫游魂卦世在四爻，应在初爻
    "归魂": {"shi": 3, "ying": 6}, # 京房八宫归魂卦世在三爻，应在上爻
}

# 六亲关系 (参照表5)
LIUQIN_RELATIONSHIPS = {
    ('木', '木'): '兄弟', ('木', '火'): '子孙', ('木', '土'): '妻财', ('木', '金'): '官鬼', ('木', '水'): '父母',
    ('火', '火'): '兄弟', ('火', '土'): '子孙', ('火', '金'): '妻财', ('火', '水'): '官鬼', ('火', '木'): '父母',
    ('土', '土'): '兄弟', ('土', '金'): '子孙', ('土', '水'): '妻财', ('土', '木'): '官鬼', ('土', '火'): '父母',
    ('金', '金'): '兄弟', ('金', '水'): '子孙', ('金', '木'): '妻财', ('金', '火'): '官鬼', ('金', '土'): '父母',
    ('水', '水'): '兄弟', ('水', '木'): '子孙', ('水', '火'): '妻财', ('水', '土'): '官鬼', ('水', '金'): '父母',
}

# 六神规则 (参照表6)
LIUSHOU_RULES = {
    '甲': ['青龙', '朱雀', '勾陈', '螣蛇', '白虎', '玄武'], '乙': ['青龙', '朱雀', '勾陈', '螣蛇', '白虎', '玄武'],
    '丙': ['朱雀', '勾陈', '螣蛇', '白虎', '玄武', '青龙'], '丁': ['朱雀', '勾陈', '螣蛇', '白虎', '玄武', '青龙'],
    '戊': ['勾陈', '螣蛇', '白虎', '玄武', '青龙', '朱雀'], '己': ['勾陈', '螣蛇', '白虎', '玄武', '青龙', '朱雀'], # 按[6]戊己同
    '庚': ['白虎', '玄武', '青龙', '朱雀', '勾陈', '螣蛇'], '辛': ['白虎', '玄武', '青龙', '朱雀', '勾陈', '螣蛇'],
    '壬': ['玄武', '青龙', '朱雀', '勾陈', '螣蛇', '白虎'], '癸': ['玄武', '青龙', '朱雀', '勾陈', '螣蛇', '白虎'],
}

# --- 辅助函数 ---
def get_trigram_from_lines(lines_tuple):
    """根据三爻二进制获取八卦名"""
    for name, info in TRIGRAMS.items():
        if info['lines'] == lines_tuple:
            return name
    return None

def lines_to_binary_str(lines_tuple):
    """将爻的元组 (0,0,0,1,1,1) 转换为二进制字符串 '111000' (上卦在前)"""
    return "".join(map(str, lines_tuple[3:] + lines_tuple[:3]))


def yao_results_to_lines(yao_res):
    """将摇卦结果 (6,7,8,9) 转换为本卦爻象 (0/1) 和动爻位置 (0-5)"""
    ben_gua_lines = []
    dong_yao_indices = []
    for i, res in enumerate(yao_res):
        if res == 6: # 老阴，本卦为阴，动爻
            ben_gua_lines.append(0)
            dong_yao_indices.append(i)
        elif res == 7: # 少阳，本卦为阳，静爻
            ben_gua_lines.append(1)
        elif res == 8: # 少阴，本卦为阴，静爻
            ben_gua_lines.append(0)
        elif res == 9: # 老阳，本卦为阳，动爻
            ben_gua_lines.append(1)
            dong_yao_indices.append(i)
        else:
            raise ValueError(f"无效的摇卦结果: {res}")
    return tuple(ben_gua_lines), dong_yao_indices

def get_bian_gua_lines(ben_gua_lines_tuple, dong_yao_indices):
    """获取变卦爻象"""
    if not dong_yao_indices:
        return None
    bian_gua_list = list(ben_gua_lines_tuple)
    for index in dong_yao_indices:
        bian_gua_list[index] = 1 - bian_gua_list[index] # 0变1, 1变0
    return tuple(bian_gua_list)

# --- 核心逻辑函数 ---
def determine_palace_info(ben_gua_lines_tuple):
    """确定卦宫信息"""
    # 将爻的顺序调整为 上卦+下卦 (高位到低位) 用于匹配BA_GONG_PALACES的键
    # 例如 (初,二,三,四,五,上) -> (上,五,四,三,二,初) -> "上五四三二初"
    # BA_GONG_PALACES 的键是 (上,五,四) + (三,二,初)
    hex_key = "".join(map(str, ben_gua_lines_tuple[::-1])) # 反转后连接，例如 (0,0,0,1,1,1) -> "111000"
    
    for palace_name, palace_data in BA_GONG_PALACES.items():
        if hex_key in palace_data['hexagrams']:
            hex_info = palace_data['hexagrams'][hex_key]
            return {
                "palace_name": palace_name,
                "palace_element": palace_data['element'],
                "hexagram_name": hex_info['name'],
                "generation_type": hex_info['generation_type']
            }
    # 如果未在简化的BA_GONG_PALACES中找到，需要完整数据或算法生成
    # 此处为简化处理，实际应用中BA_GONG_PALACES应包含所有64卦
    # 或者实现京房八宫的爻变算法来动态确定
    # 爻变算法：
    # 1. 找到纯卦
    # 2. 逐爻变，与目标卦比较，确定世数
    # 3. 若非1-5世，则从5世卦变第四爻为游魂，再变内卦为归魂
    # 这是一个复杂过程，此处暂用查表法（假设BA_GONG_PALACES已完备）
    raise ValueError(f"未能在八宫数据中找到卦象: {hex_key}. 请确保BA_GONG_PALACES数据完整。")


def assign_na_jia(ben_gua_lines_tuple, palace_info_result):
    """为六爻分配纳甲干支"""
    # 1. 确定内外卦的纯卦形式
    # ben_gua_lines_tuple 是 (初,二,三,四,五,上)
    inner_trigram_lines = ben_gua_lines_tuple[0:3]
    outer_trigram_lines = ben_gua_lines_tuple[3:6]

    inner_trigram_name = get_trigram_from_lines(inner_trigram_lines)
    outer_trigram_name = get_trigram_from_lines(outer_trigram_lines)

    if not inner_trigram_name or not outer_trigram_name:
        raise ValueError("无法识别内外卦")

    # 2. 根据内外卦的纯卦形式，从NAJIA_RULES获取其纳甲
    # 注意：NAJIA_RULES中的干支是针对纯卦的六个爻的
    # 对于组合卦，内卦取其纯卦的前三爻纳甲，外卦取其纯卦的后三爻纳甲
    
    na_jia_lines = []
    # 内卦 (初、二、三爻)
    inner_najia_rules = NAJIA_RULES[inner_trigram_name]
    for i in range(3):
        na_jia_lines.append(inner_najia_rules[i])
    
    # 外卦 (四、五、上爻)
    outer_najia_rules = NAJIA_RULES[outer_trigram_name]
    for i in range(3, 6):
        na_jia_lines.append(outer_najia_rules[i])
        
    return na_jia_lines


def locate_shi_ying(generation_type):
    """确定世应爻位置"""
    if generation_type in SHI_YING_RULES:
        return SHI_YING_RULES[generation_type]
    raise ValueError(f"未知的世序类型: {generation_type}")

def assign_liu_qin(na_jia_assigned_lines, palace_element):
    """分配六亲"""
    liu_qin_lines = []
    for _, dizhi in na_jia_assigned_lines:
        yao_element = DIZHI_INFO[dizhi]['element']
        relation_key = (palace_element, yao_element) # 注意顺序：(宫五行, 爻五行)
        
        # 调整为 (爻五行, 宫五行) 来匹配 LIUQIN_RELATIONSHIPS 的生克逻辑
        # 例如：爻木，宫金。关系是金克木（官鬼克我），所以爻是“妻财”（我克者）相对于宫。
        # 不对，LIUQIN_RELATIONSHIPS 的键应该是 (卦宫五行, 爻支五行)
        # 生我者为父母：爻生宫 -> 父母
        # 我生者为子孙：宫生爻 -> 子孙
        
        # 重新梳理六亲逻辑：
        # 爻五行 生 宫五行 => 父母 (爻是父母)
        # 宫五行 生 爻五行 => 子孙 (爻是子孙)
        # 爻五行 同 宫五行 => 兄弟 (爻是兄弟)
        # 爻五行 克 宫五行 => 妻财 (爻是妻财)
        # 宫五行 克 爻五行 => 官鬼 (爻是官鬼)

        # 使用标准的五行生克判断
        if yao_element == palace_element:
            liu_qin = "兄弟"
        # 宫生爻
        elif (palace_element == '木' and yao_element == '火') or \
             (palace_element == '火' and yao_element == '土') or \
             (palace_element == '土' and yao_element == '金') or \
             (palace_element == '金' and yao_element == '水') or \
             (palace_element == '水' and yao_element == '木'):
            liu_qin = "子孙"
        # 爻生宫
        elif (yao_element == '木' and palace_element == '火') or \
             (yao_element == '火' and palace_element == '土') or \
             (yao_element == '土' and palace_element == '金') or \
             (yao_element == '金' and palace_element == '水') or \
             (yao_element == '水' and palace_element == '木'):
            liu_qin = "父母"
        # 宫克爻
        elif (palace_element == '木' and yao_element == '土') or \
             (palace_element == '火' and yao_element == '金') or \
             (palace_element == '土' and yao_element == '水') or \
             (palace_element == '金' and yao_element == '木') or \
             (palace_element == '水' and yao_element == '火'):
            liu_qin = "官鬼" # 爻被宫克
        # 爻克宫
        elif (yao_element == '木' and palace_element == '土') or \
             (yao_element == '火' and palace_element == '金') or \
             (yao_element == '土' and palace_element == '水') or \
             (yao_element == '金' and palace_element == '木') or \
             (yao_element == '水' and palace_element == '火'):
            liu_qin = "妻财" # 爻克宫
        else:
            liu_qin = "错误" # 不应发生
        liu_qin_lines.append(liu_qin)
    return liu_qin_lines


def assign_liu_shou(day_tiangan_char):
    """分配六神"""
    if day_tiangan_char in LIUSHOU_RULES:
        return LIUSHOU_RULES[day_tiangan_char]
    raise ValueError(f"无效的日天干: {day_tiangan_char}")

# --- 主装卦函数 ---
def zhuang_gua(yao_results_list, day_tiangan_char):
    """
    进行完整的装卦过程
    yao_results_list: 六次摇卦结果列表, e.g.,  (初爻在前)
    day_tiangan_char: 当日天干字符, e.g., '甲'
    """
    if len(yao_results_list)!= 6:
        raise ValueError("摇卦结果必须是6个爻")
    if day_tiangan_char not in TIANGAN:
        raise ValueError(f"无效的日天干: {day_tiangan_char}")

    ben_gua_lines, dong_yao_indices = yao_results_to_lines(yao_results_list)
    print(f"{ben_gua_lines=}")
    print(f"{dong_yao_indices=}")
    # 1. 确定卦宫信息
    # 注意：palace_info中的卦名是基于标准64卦名，可能与BA_GONG_PALACES中的略有差异（如“乾为天”）
    # ben_gua_lines是 (初,二,三,四,五,上)
    palace_info = determine_palace_info(ben_gua_lines)

    # 2. 纳甲
    na_jia_assigned = assign_na_jia(ben_gua_lines, palace_info)

    # 3. 世应
    shi_ying_pos = locate_shi_ying(palace_info['generation_type'])

    # 4. 六亲
    liu_qin_assigned = assign_liu_qin(na_jia_assigned, palace_info['palace_element'])

    # 5. 六神
    liu_shou_assigned = assign_liu_shou(day_tiangan_char)

    # 6. 变卦 (如果存在动爻)
    bian_gua_info = None
    if dong_yao_indices:
        bian_gua_lines_tuple = get_bian_gua_lines(ben_gua_lines, dong_yao_indices)
        # 对变卦也可以进行完整的装卦，但通常只关心其卦象和爻辞
        # 此处简化，只获取变卦卦名和爻象
        bian_palace_info = determine_palace_info(bian_gua_lines_tuple) # 获取变卦名
        bian_gua_info = {
            "lines_tuple": bian_gua_lines_tuple,
            "hex_name": bian_palace_info['hexagram_name'], # 使用palace_info获取标准卦名
            "lines_char": [('—' if y == 1 else '--') for y in bian_gua_lines_tuple]
        }

    # 组织结果
    result = {
        "ben_gua": {
            "lines_tuple": ben_gua_lines,
            "hex_name": palace_info['hexagram_name'],
            "lines_char": [('—' if y == 1 else '--') for y in ben_gua_lines],
            "palace": palace_info['palace_name'],
            "palace_element": palace_info['palace_element'],
            "generation_type": palace_info['generation_type'],
            "dong_yao_indices": dong_yao_indices, # 0-indexed
            "yaos": []
            
        },
        "bian_gua": bian_gua_info
    }

    yao_labels = ["初爻", "二爻", "三爻", "四爻", "五爻", "上爻"]
    for i in range(6):
        yao_data = {
            "label": yao_labels[i],
            "line_char": ('●' if ben_gua_lines[i] == 1 else 'X') if i in dong_yao_indices else ('—' if ben_gua_lines[i] == 1 else '--'),
            "tiangan": na_jia_assigned[i],
            "dizhi": na_jia_assigned[i],
            "dizhi_element": DIZHI_INFO[na_jia_assigned[i]]['element'],
            "liu_qin": liu_qin_assigned[i],
            "liu_shou": liu_shou_assigned[i],
            "is_shi": (i + 1) == shi_ying_pos['shi'],
            "is_ying": (i + 1) == shi_ying_pos['ying'],
            "is_dong": i in dong_yao_indices
        }
        result["ben_gua"]["yaos"].append(yao_data)
        
    return result

# --- 输出显示 ---
def display_gua_info(gua_data):
    """以可读格式显示装卦结果"""
    ben = gua_data["ben_gua"]
    print(f"本卦: {ben['hex_name']} ({''.join(ben['lines_char'])})")
    print(f"所属卦宫: {ben['palace']} ({ben['palace_element']}宫)")
    print(f"世序: {ben['generation_type']}")
    print("-" * 30)
    for i in range(5, -1, -1): # 从上爻开始打印
        yao = ben["yaos"][i]
        shi_ying_mark = ""
        if yao["is_shi"]: shi_ying_mark = " (世)"
        if yao["is_ying"]: shi_ying_mark = " (应)"
        
        line_display = yao["line_char"]
        
        print(f"{yao['label']}: {line_display}  {yao['tiangan']}{yao['dizhi']}({yao['dizhi_element']})  {yao['liu_qin']}  {yao['liu_shou']}{shi_ying_mark}")
    
    if gua_data["bian_gua"]:
        bian = gua_data["bian_gua"]
        print("-" * 30)
        print(f"变卦: {bian['hex_name']} ({''.join(bian['lines_char'])})")

In [15]:
# 示例：
# 假设起卦结果 (初爻到上爻): 
# 假设当日天干: '甲'

# 补全 BA_GONG_PALACES 和 HEXAGRAM_NAMES 才能完整运行
# 例如，为 乾宫 添加 天风姤 (011111) 等
# 此处假设 BA_GONG_PALACES 和 HEXAGRAM_NAMES 已有 '天风姤' (011111) 的定义：
# 在 BA_GONG_PALACES['乾']['hexagrams'] 中加入:
# "011111": {"name": "天风姤", "generation_type": "一世"},
# 在 HEXAGRAM_NAMES 中加入:
# "011111": "天风姤", 
# 变卦为 火风鼎 (011101)
# "011101": "火风鼎",
# BA_GONG_PALACES['离']['hexagrams'] (假设离为火)
# "011101": {"name": "火风鼎", "generation_type": "某世"}, # 需要确定鼎卦在离宫的世序

# 示例：乾宫一世卦 天风姤
# 摇卦结果：初爻动，其他静爻。假设初爻为9，其余为7或8，使得本卦为天风姤。
# 天风姤 (乾上巽下): 下卦巽 (110), 上卦乾 (111) -> (0,1,1,1,1,1) (初爻在左)
# 对应摇卦结果:  (初爻为9，变成0，即巽卦初爻)
yao_input =  [7,7,7,7,7,7]# 得到 天风姤，初爻动
day_stem = '乙'

# 为了能运行，需要确保 BA_GONG_PALACES['乾']['hexagrams'] 中有 "011111" (天风姤)
# 变卦是 火风鼎 (011101) -> 巽下离上
# 需要确保 BA_GONG_PALACES 某个宫 (如离宫) 包含 火风鼎 "011101"
# 假设已补全数据：
if "011111" not in BA_GONG_PALACES['乾']['hexagrams']:
    BA_GONG_PALACES['乾']['hexagrams']["011111"] = {"name": "天风姤", "generation_type": "一世"}
if "011101" not in HEXAGRAM_NAMES: # 假设火风鼎在离宫，是二世卦
     HEXAGRAM_NAMES["011101"] = "火风鼎"
     if '离' not in BA_GONG_PALACES: # 假设离宫数据
         BA_GONG_PALACES['离'] = {'head_trigram': '离', 'element': '火', 'hexagrams': {
             "101101": {"name": "离为火", "generation_type": "八纯"},
             "001101": {"name": "火山旅", "generation_type": "一世"},
             "101101": {"name": "火风鼎", "generation_type": "二世"} # 假设
         }}
     elif "011101" not in BA_GONG_PALACES['离']['hexagrams']:
          BA_GONG_PALACES['离']['hexagrams']["011101"] = {"name": "火风鼎", "generation_type": "二世"}


try:
    gua_result = zhuang_gua(yao_input, day_stem)
    display_gua_info(gua_result)
except ValueError as e:
    print(f"发生错误: {e}")
    print("请确保 BA_GONG_PALACES 和 HEXAGRAM_NAMES 数据已针对此卦例补全。")

ben_gua_lines=(1, 1, 1, 1, 1, 1)
dong_yao_indices=[]


KeyError: ('甲', '子')

In [26]:
all_gua_data = {
    '乾为天': {
        'gong': '乾宫',
        'yao': '000000', # 上爻到初爻
        'yao_info': [
            {'name': '父母戌土 世'},
            {'name': '兄弟申金'},
            {'name': '官鬼午火'},
            {'name': '父母辰土 应'},
            {'name': '妻财寅木'},
            {'name': '子孙子水'}
        ]
    },
    '天风姤': {
        'gong': '乾宫',
        'yao': '000001',
        'yao_info': [
            {'name': '父母戌土'},
            {'name': '兄弟申金'},
            {'name': '官鬼午火 应'},
            {'name': '兄弟酉金'},
            {'name': '子孙亥水'},
            {'name': '父母丑土 世'}
        ]
    },
    '天山遁': {
        'gong': '乾宫',
        'yao': '000011',
        'yao_info': [
            {'name': '父母戌土'},
            {'name': '兄弟申金 应'},
            {'name': '官鬼午火'},
            {'name': '兄弟申金'},
            {'name': '官鬼午火 世'},
            {'name': '父母辰土'}
        ]
    },
    '天地否': {
        'gong': '乾宫',
        'yao': '000111',
        'yao_info': [
            {'name': '父母戌土 应'},
            {'name': '兄弟申金'},
            {'name': '官鬼午火'},
            {'name': '妻财卯木 世'},
            {'name': '官鬼巳火'},
            {'name': '父母未土'}
        ]
    },
    '风地观': {
        'gong': '乾宫',
        'yao': '001111',
        'yao_info': [
            {'name': '妻财卯木'},
            {'name': '官鬼巳火'},
            {'name': '父母未土 世'},
            {'name': '妻财卯木'},
            {'name': '官鬼巳火 应'},
            {'name': '父母未土'}
        ]
    },
    '山地剥': {
        'gong': '乾宫',
        'yao': '011111',
        'yao_info': [
            {'name': '妻财寅木'},
            {'name': '子孙子水 世'},
            {'name': '父母戌土'},
            {'name': '妻财卯木 应'},
            {'name': '官鬼巳火'},
            {'name': '父母未土'}
        ]
    },
    '火地晋': {
        'gong': '乾宫',
        'yao': '010111',
        'yao_info': [
            {'name': '官鬼巳火'},
            {'name': '父母未土'},
            {'name': '兄弟酉金 世'},
            {'name': '妻财卯木'},
            {'name': '官鬼巳火 应'},
            {'name': '父母未土'}
        ]
    },
    '火天大有': {
        'gong': '乾宫',
        'yao': '010000',
        'yao_info': [
            {'name': '官鬼巳火 应'},
            {'name': '父母未土'},
            {'name': '兄弟酉金'},
            {'name': '父母辰土 世'},
            {'name': '妻财寅木'},
            {'name': '子孙子水'}
        ]
    },
    '兑为泽': {
        'gong': '兑宫',
        'yao': '100100',
        'yao_info': [
            {'name': '父母未土 世'},
            {'name': '兄弟酉金'},
            {'name': '子孙亥水'},
            {'name': '父母丑土 应'},
            {'name': '妻财卯木'},
            {'name': '官鬼巳火'}
        ]
    },
    '泽水困': {
        'gong': '兑宫',
        'yao': '100001',
        'yao_info': [
            {'name': '父母未土'},
            {'name': '兄弟酉金'},
            {'name': '子孙亥水 应'},
            {'name': '官鬼午火'},
            {'name': '父母辰土'},
            {'name': '妻财寅木 世'}
        ]
    },
    '泽地萃': {
        'gong': '兑宫',
        'yao': '100111',
        'yao_info': [
            {'name': '父母未土'},
            {'name': '兄弟酉金 应'},
            {'name': '子孙亥水'},
            {'name': '妻财卯木'},
            {'name': '官鬼巳火 世'},
            {'name': '父母未土'}
        ]
    },
    '泽山咸': {
        'gong': '兑宫',
        'yao': '100011',
        'yao_info': [
            {'name': '父母未土 应'},
            {'name': '兄弟酉金'},
            {'name': '子孙亥水'},
            {'name': '兄弟申金 世'},
            {'name': '官鬼午火'},
            {'name': '父母辰土'}
        ]
    },
    '水山蹇': {
        'gong': '兑宫',
        'yao': '101011',
        'yao_info': [
            {'name': '子孙子水'},
            {'name': '父母戌土'},
            {'name': '兄弟申金 世'},
            {'name': '兄弟申金'},
            {'name': '官鬼午火'},
            {'name': '父母辰土 应'}
        ]
    },
    '地山谦': {
        'gong': '兑宫',
        'yao': '111011',
        'yao_info': [
            {'name': '兄弟酉金'},
            {'name': '子孙亥水 世'},
            {'name': '父母丑土'},
            {'name': '兄弟申金'},
            {'name': '官鬼午火 应'},
            {'name': '父母辰土'}
        ]
    },
    '雷山小过': {
        'gong': '兑宫',
        'yao': '110011',
        'yao_info': [
            {'name': '父母戌土'},
            {'name': '兄弟申金'},
            {'name': '官鬼午火 世'},
            {'name': '兄弟申金'},
            {'name': '官鬼午火'},
            {'name': '父母辰土 应'}
        ]
    },
    '雷泽归妹': {
        'gong': '兑宫',
        'yao': '110100',
        'yao_info': [
            {'name': '父母戌土 应'},
            {'name': '兄弟申金'},
            {'name': '官鬼午火'},
            {'name': '父母丑土 世'},
            {'name': '妻财卯木'},
            {'name': '官鬼巳火'}
        ]
    },
    '离为火': {
        'gong': '离宫',
        'yao': '010010',
        'yao_info': [
            {'name': '兄弟巳火 世'},
            {'name': '子孙未土'},
            {'name': '妻财酉金'},
            {'name': '官鬼亥水 应'},
            {'name': '子孙丑土'},
            {'name': '父母卯木'}
        ]
    },
    '火山旅': {
        'gong': '离宫',
        'yao': '010100',
        'yao_info': [
            {'name': '兄弟巳火'},
            {'name': '子孙未土'},
            {'name': '妻财酉金 应'},
            {'name': '妻财申金'},
            {'name': '兄弟午火'},
            {'name': '子孙辰土 世'}
        ]
    },
    '火风鼎': {
        'gong': '离宫',
        'yao': '010101',
        'yao_info': [
            {'name': '兄弟巳火'},
            {'name': '子孙未土 应'},
            {'name': '妻财酉金'},
            {'name': '妻财酉金'},
            {'name': '官鬼亥水 世'},
            {'name': '父母卯木'}
        ]
    },
    '火水未济': {
        'gong': '离宫',
        'yao': '010110',
        'yao_info': [
            {'name': '兄弟巳火 应'},
            {'name': '子孙未土'},
            {'name': '妻财酉金'},
            {'name': '官鬼亥水 世'},
            {'name': '子孙丑土'},
            {'name': '父母卯木'}
        ]
    },
    '山水蒙': {
        'gong': '离宫',
        'yao': '011110', # Note: Image has 父母丙火 (Red/0) as top line
        'yao_info': [
            {'name': '父母丙火'},
            {'name': '官鬼子水'}, # Image text showed 官鬼子水
            {'name': '子孙戌土 世'},
            {'name': '兄弟午火'},
            {'name': '子孙辰土 应'},
            {'name': '父母寅木'}
        ]
    },
    '风水涣': {
        'gong': '离宫',
        'yao': '001110', # 巽上坎下 001 101
        'yao_info': [
            {'name': '父母卯木'},
            {'name': '兄弟巳火 世'},
            {'name': '子孙未土'},
            {'name': '兄弟午火 应'}, # Note: Image has 兄弟午火 应 (Red/0)
            {'name': '子孙辰土'},   # Note: Image has 子孙辰土 (Red/0)
            {'name': '父母寅木'}    # Note: Image has 父母寅木 (Blue/1) - Corrected yao string: 001001 for 风水涣
        ]
    },
    # Correcting 风水涣 based on re-check of typical structure and trying to reconcile with image if possible
    # 风(巽 001) 水(坎 101) -> 001101
    # Image text: 父卯(0) 兄巳(0)世 子未(1) || 兄午(0)应 子辰(0) 父寅(1)
    # Yao string from image text: 001001
    '风水涣': { # Re-corrected based on image interpretation
        'gong': '离宫',
        'yao': '001001',
        'yao_info': [
            {'name': '父母卯木'},
            {'name': '兄弟巳火 世'},
            {'name': '子孙未土'},
            {'name': '兄弟午火 应'},
            {'name': '子孙辰土'},
            {'name': '父母寅木'}
        ]
    },
    '天水讼': {
        'gong': '离宫',
        'yao': '000110', # 乾上坎下 000 101
        'yao_info': [
            {'name': '子孙戌土'},
            {'name': '妻财申金'},
            {'name': '兄弟午火 世'},
            {'name': '兄弟午火'}, # Note: Image has 兄弟午火 (Red/0)
            {'name': '子孙辰土 应'},# Note: Image has 子孙辰土 (Red/0)
            {'name': '父母寅木'}   # Note: Image has 父母寅木 (Blue/1) - Corrected yao string: 000001 for 天水讼
        ]
    },
    # Correcting 天水讼 based on re-check
    # Image text: 子戌(0) 才申(0) 兄午(0)世 || 兄午(0) 子辰(0)应 父寅(1)
    # Yao string from image text: 000001
    '天水讼': { # Re-corrected based on image interpretation
        'gong': '离宫',
        'yao': '000001',
        'yao_info': [
            {'name': '子孙戌土'},
            {'name': '妻财申金'},
            {'name': '兄弟午火 世'},
            {'name': '兄弟午火'},
            {'name': '子孙辰土 应'},
            {'name': '父母寅木'}
        ]
    },
    '天火同人': {
        'gong': '离宫',
        'yao': '000010',
        'yao_info': [
            {'name': '子孙戌土 应'},
            {'name': '妻财申金'},
            {'name': '兄弟午火'},
            {'name': '官鬼亥水 世'},
            {'name': '子孙丑土'},
            {'name': '父母卯木'}
        ]
    },
    '震为雷': {
        'gong': '震宫',
        'yao': '100100', #震卦本身
        'yao_info': [
            {'name': '妻财戌土 世'},
            {'name': '官鬼申金'},
            {'name': '子孙午火'},
            {'name': '妻财辰土 应'},
            {'name': '兄弟寅木'},
            {'name': '父母子水'}
        ]
    },
    '雷地豫': {
        'gong': '震宫',
        'yao': '100111',
        'yao_info': [
            {'name': '妻财戌土'},
            {'name': '官鬼申金'},
            {'name': '子孙午火 应'},
            {'name': '兄弟卯木'},
            {'name': '子孙巳火'},
            {'name': '妻财未土 世'}
        ]
    },
    '雷水解': {
        'gong': '震宫',
        'yao': '100101',
        'yao_info': [
            {'name': '妻财戌土'},
            {'name': '官鬼申金 应'},
            {'name': '子孙午火'},
            {'name': '子孙午火'},
            {'name': '妻财辰土 世'},
            {'name': '兄弟寅木'}
        ]
    },
    '雷风恒': {
        'gong': '震宫',
        'yao': '100001',
        'yao_info': [
            {'name': '妻财戌土 应'},
            {'name': '官鬼申金'},
            {'name': '子孙午火'},
            {'name': '官鬼酉金 世'},
            {'name': '父母亥水'},
            {'name': '妻财丑土'}
        ]
    },
    '地风升': {
        'gong': '震宫',
        'yao': '111001',
        'yao_info': [
            {'name': '官鬼酉金'},
            {'name': '父母亥水'},
            {'name': '妻财丑土 世'},
            {'name': '官鬼酉金'},
            {'name': '父母亥水 应'},
            {'name': '妻财丑土'}
        ]
    },
    '水风井': {
        'gong': '震宫',
        'yao': '101001',
        'yao_info': [
            {'name': '父母子水'},
            {'name': '妻财戌土 世'},
            {'name': '官鬼申金'},
            {'name': '官鬼酉金 应'},
            {'name': '父母亥水'},
            {'name': '妻财丑土'}
        ]
    },
    '泽风大过': {
        'gong': '震宫',
        'yao': '100001',
        'yao_info': [
            {'name': '妻财未土'},
            {'name': '官鬼酉金'},
            {'name': '父母亥水 世'},
            {'name': '官鬼酉金'},
            {'name': '父母亥水 应'},
            {'name': '妻财丑土'}
        ]
    },
    '泽雷随': {
        'gong': '震宫',
        'yao': '100100', # Duplicates 震为雷, but it's 兑上震下 (100100)
        'yao_info': [
            {'name': '妻财未土 应'},
            {'name': '官鬼酉金'},
            {'name': '父母亥水'},
            {'name': '妻财辰土 世'},
            {'name': '兄弟寅木'},
            {'name': '父母子水'}
        ]
    },
    '巽为风': {
        'gong': '巽宫',
        'yao': '001001',
        'yao_info': [
            {'name': '兄弟卯木 世'},
            {'name': '子孙巳火'},
            {'name': '妻财未土'},
            {'name': '官鬼酉金 应'},
            {'name': '父母亥水'},
            {'name': '妻财丑土'}
        ]
    },
    '风天小畜': {
        'gong': '巽宫',
        'yao': '001000',
        'yao_info': [
            {'name': '兄弟卯木'},
            {'name': '子孙巳火'},
            {'name': '妻财未土 应'},
            {'name': '妻财辰土'},
            {'name': '兄弟寅木'},
            {'name': '子孙子水 世'}
        ]
    },
    '风火家人': {
        'gong': '巽宫',
        'yao': '001010',
        'yao_info': [
            {'name': '兄弟卯木'},
            {'name': '子孙巳火 应'},
            {'name': '妻财未土'},
            {'name': '父母亥水'},
            {'name': '妻财丑土 世'},
            {'name': '兄弟卯木'}
        ]
    },
    '风雷益': {
        'gong': '巽宫',
        'yao': '001100',
        'yao_info': [
            {'name': '兄弟卯木 应'},
            {'name': '子孙巳火'},
            {'name': '妻财未土'},
            {'name': '妻财辰土 世'},
            {'name': '兄弟寅木'},
            {'name': '父母子水'}
        ]
    },
    '天雷无妄': {
        'gong': '巽宫',
        'yao': '000100',
        'yao_info': [
            {'name': '妻财戌土'},
            {'name': '官鬼申金'},
            {'name': '子孙午火 世'},
            {'name': '妻财辰土'},
            {'name': '兄弟寅木 应'},
            {'name': '父母子水'}
        ]
    },
    '火雷噬嗑': {
        'gong': '巽宫',
        'yao': '010100',
        'yao_info': [
            {'name': '子孙巳火'},
            {'name': '妻财未土 世'},
            {'name': '官鬼酉金'},
            {'name': '妻财辰土 应'},
            {'name': '兄弟寅木'},
            {'name': '父母子水'}
        ]
    },
    '山雷颐': {
        'gong': '巽宫',
        'yao': '011100',
        'yao_info': [
            {'name': '兄弟寅木'},
            {'name': '父母子水'},
            {'name': '妻财戌土 世'},
            {'name': '妻财辰土'},
            {'name': '兄弟寅木 应'},
            {'name': '父母子水'}
        ]
    },
    '山风蛊': {
        'gong': '巽宫',
        'yao': '011001',
        'yao_info': [
            {'name': '兄弟寅木 应'},
            {'name': '父母子水'},
            {'name': '妻财戌土'},
            {'name': '官鬼酉金 世'},
            {'name': '父母亥水'},
            {'name': '妻财丑土'}
        ]
    },
    '坎为水': {
        'gong': '坎宫',
        'yao': '101101',
        'yao_info': [
            {'name': '兄弟子水 世'},
            {'name': '官鬼戌土'},
            {'name': '父母申金'},
            {'name': '妻财午火 应'},
            {'name': '官鬼辰土'},
            {'name': '子孙寅木'}
        ]
    },
    '水泽节': {
        'gong': '坎宫',
        'yao': '101100',
        'yao_info': [
            {'name': '兄弟子水'},
            {'name': '官鬼戌土'},
            {'name': '父母申金 应'},
            {'name': '官鬼丑土'},
            {'name': '子孙卯木'},
            {'name': '妻财巳火 世'}
        ]
    },
    '水雷屯': {
        'gong': '坎宫',
        'yao': '101100', # Note: Yao string same as 水泽节 due to image transcription.
        'yao_info': [
            {'name': '兄弟子水'},
            {'name': '官鬼戌土'},
            {'name': '父母申金 应'},
            {'name': '官鬼辰土'},
            {'name': '子孙寅木 世'},
            {'name': '兄弟子水'} # From image for 水雷屯 bottom line.
        ]
    },
    '水火既济': {
        'gong': '坎宫',
        'yao': '101010',
        'yao_info': [
            {'name': '兄弟子水 应'},
            {'name': '官鬼戌土'},
            {'name': '父母申金'},
            {'name': '兄弟亥水 世'},
            {'name': '官鬼丑土'},
            {'name': '妻财巳火'}
        ]
    },
    '泽火革': {
        'gong': '坎宫',
        'yao': '100010',
        'yao_info': [
            {'name': '官鬼未土'},
            {'name': '父母酉金'},
            {'name': '兄弟亥水 世'},
            {'name': '兄弟亥水'},
            {'name': '官鬼丑土 应'},
            {'name': '妻财巳火'}
        ]
    },
    '雷火丰': {
        'gong': '坎宫',
        'yao': '100010', # Note: Yao string same as 泽火革.
        'yao_info': [
            {'name': '官鬼戌土'},
            {'name': '父母申金'},
            {'name': '妻财午火 世'},
            {'name': '兄弟亥水'},
            {'name': '官鬼丑土 应'},
            {'name': '子孙卯木'}
        ]
    },
    '地火明夷': {
        'gong': '坎宫',
        'yao': '111010',
        'yao_info': [
            {'name': '父母酉金'},
            {'name': '兄弟亥水'},
            {'name': '官鬼丑土 世'},
            {'name': '兄弟亥水'},
            {'name': '官鬼丑土 应'},
            {'name': '妻财巳火'}
        ]
    },
    '地水师': {
        'gong': '坎宫',
        'yao': '111101',
        'yao_info': [
            {'name': '父母酉金 应'},
            {'name': '兄弟亥水'},
            {'name': '官鬼丑土'},
            {'name': '妻财午火 世'},
            {'name': '官鬼辰土'},
            {'name': '子孙寅木'}
        ]
    },
    '艮为山': {
        'gong': '艮宫',
        'yao': '011011',
        'yao_info': [
            {'name': '官鬼寅木 世'},
            {'name': '妻财子水'},
            {'name': '兄弟戌土'},
            {'name': '子孙申金 应'},
            {'name': '父母午火'},
            {'name': '兄弟辰土'}
        ]
    },
    '山火贲': {
        'gong': '艮宫',
        'yao': '011010',
        'yao_info': [
            {'name': '官鬼寅木'},
            {'name': '妻财子水'},
            {'name': '兄弟戌土 应'},
            {'name': '妻财亥水'},
            {'name': '兄弟丑土'},
            {'name': '官鬼卯木 世'}
        ]
    },
    '山天大畜': {
        'gong': '艮宫',
        'yao': '011000',
        'yao_info': [
            {'name': '官鬼寅木'},
            {'name': '妻财子水 应'},
            {'name': '兄弟戌土'},
            {'name': '兄弟辰土'},
            {'name': '官鬼寅木 世'},
            {'name': '妻财子水'}
        ]
    },
    '山泽损': {
        'gong': '艮宫',
        'yao': '011100',
        'yao_info': [
            {'name': '官鬼寅木 应'},
            {'name': '妻财子水'},
            {'name': '兄弟戌土'},
            {'name': '兄弟丑土 世'},
            {'name': '官鬼卯木'},
            {'name': '父母巳火'}
        ]
    },
    '火泽睽': {
        'gong': '艮宫',
        'yao': '010100',
        'yao_info': [
            {'name': '父母巳火'},
            {'name': '兄弟未土'},
            {'name': '子孙酉金 世'},
            {'name': '兄弟丑土'},
            {'name': '官鬼卯木 应'},
            {'name': '父母巳火'}
        ]
    },
    '天泽履': {
        'gong': '艮宫',
        'yao': '000100',
        'yao_info': [
            {'name': '兄弟戌土'},
            {'name': '子孙申金 世'},
            {'name': '父母午火'},
            {'name': '兄弟丑土 应'},
            {'name': '官鬼卯木'},
            {'name': '父母巳火'}
        ]
    },
    '风泽中孚': {
        'gong': '艮宫',
        'yao': '001100',
        'yao_info': [
            {'name': '官鬼卯木'},
            {'name': '父母巳火'},
            {'name': '兄弟未土 世'},
            {'name': '兄弟丑土'},
            {'name': '官鬼卯木 应'},
            {'name': '父母巳火'}
        ]
    },
    '风山渐': {
        'gong': '艮宫',
        'yao': '001011',
        'yao_info': [
            {'name': '官鬼卯木 应'},
            {'name': '父母巳火'},
            {'name': '兄弟未土'},
            {'name': '子孙申金 世'},
            {'name': '父母午火'},
            {'name': '兄弟辰土'}
        ]
    },
    '坤为地': {
        'gong': '坤宫',
        'yao': '111111',
        'yao_info': [
            {'name': '子孙酉金 世'},
            {'name': '妻财亥水'},
            {'name': '兄弟丑土'},
            {'name': '官鬼卯木 应'},
            {'name': '父母巳火'},
            {'name': '兄弟未土'}
        ]
    },
    '地雷复': {
        'gong': '坤宫',
        'yao': '111100',
        'yao_info': [
            {'name': '子孙酉金'},
            {'name': '妻财亥水'},
            {'name': '兄弟丑土 应'},
            {'name': '兄弟辰土'},
            {'name': '官鬼寅木'},
            {'name': '妻财子水 世'}
        ]
    },
    '地泽临': {
        'gong': '坤宫',
        'yao': '111100', # Note: Yao string same as 地雷复.
        'yao_info': [
            {'name': '子孙酉金'},
            {'name': '妻财亥水 应'},
            {'name': '兄弟丑土'},
            {'name': '兄弟丑土'},
            {'name': '官鬼卯木 世'},
            {'name': '父母巳火'}
        ]
    },
    '地天泰': {
        'gong': '坤宫',
        'yao': '111000',
        'yao_info': [
            {'name': '子孙酉金 应'},
            {'name': '妻财亥水'},
            {'name': '兄弟丑土'},
            {'name': '兄弟辰土 世'},
            {'name': '官鬼寅木'},
            {'name': '妻财子水'}
        ]
    },
    '雷天大壮': {
        'gong': '坤宫',
        'yao': '100000',
        'yao_info': [
            {'name': '兄弟戌土'},
            {'name': '子孙申金'},
            {'name': '父母午火 世'},
            {'name': '兄弟辰土'},
            {'name': '官鬼寅木 应'},
            {'name': '妻财子水'}
        ]
    },
    '泽天夬': {
        'gong': '坤宫',
        'yao': '100000', # Note: Yao string same as 雷天大壮.
        'yao_info': [
            {'name': '兄弟未土'},
            {'name': '子孙酉金 世'},
            {'name': '妻财亥水'},
            {'name': '兄弟辰土 应'},
            {'name': '官鬼寅木'},
            {'name': '妻财子水'}
        ]
    },
    '水天需': {
        'gong': '坤宫',
        'yao': '101000',
        'yao_info': [
            {'name': '妻财子水'},
            {'name': '兄弟戌土'},
            {'name': '子孙申金 世'},
            {'name': '兄弟辰土'},
            {'name': '官鬼寅木 应'},
            {'name': '妻财子水'} # Note: Bottom line same as top line name based on image.
        ]
    },
    '水地比': {
        'gong': '坤宫',
        'yao': '101111',
        'yao_info': [
            {'name': '妻财子水 应'},
            {'name': '兄弟戌土'},
            {'name': '子孙申金'},
            {'name': '官鬼卯木 世'},
            {'name': '父母巳火'},
            {'name': '兄弟未土'}
        ]
    }
}


current_gua_name = '乾为天'
if current_gua_name in all_gua_data:
    data = all_gua_data[current_gua_name]
    gong = data['gong']
    yao_str = data['yao']
    yao_info_list = data['yao_info']

    print(f"卦名: {current_gua_name}")
    print(f"所属宫位: {gong}")
    print(f"爻象 (上爻到初爻, 0为阳, 1为阴): {yao_str}")
    print("爻辞信息 (从上爻到初爻):")
    for i, yao_detail in enumerate(yao_info_list):
        print(f"  {['上','五','四','三','二','初'][i]}爻: {yao_detail['name']}")

    # 接下来可以将这些数据传递给您的 format_gua 函数
    # 注意调整 format_gua 函数中对 yao_info 内字典的访问方式，
    # 因为现在它只包含 {'name': ...}

    # 您的 format_gua 函数可能需要如下的 ben_gua_input 和 ben_yao_info_input:
    # ben_gua_input = {
    #     'gong': gong,
    #     'name': current_gua_name, # 您的函数可能需要这个name
    #     'yao': yao_str
    # }
    # # 您的原始 format_gua 函数期望 ben_yao_info 的每个元素有 'liu_shen' 和 'name'
    # # 您可能需要动态添加 'liu_shen': '' 和 'dong': False (如果还需要这些字段)
    # # 或者修改 format_gua 以适应新的简化版 yao_info
    # ben_yao_info_input_for_func = []
    # for item in yao_info_list:
    #    ben_yao_info_input_for_func.append({
    #        'liu_shen': '', # 或实际的六神
    #        'name': item['name'],
    #        'dong': False # 或实际的动爻状态
    #    })

else:
    print(f"未找到卦 [{current_gua_name}] 的数据")

卦名: 乾为天
所属宫位: 乾宫
爻象 (上爻到初爻, 0为阳, 1为阴): 000000
爻辞信息 (从上爻到初爻):
  上爻: 父母戌土 世
  五爻: 兄弟申金
  四爻: 官鬼午火
  三爻: 父母辰土 应
  二爻: 妻财寅木
  初爻: 子孙子水
